# SNAC 
### Spiking Neural Autoencoder for time series Compression

This is a main script to test out the idea of Spiking neural networks compression of time series of sensor reads before packing up everyithing in a nice library.

First things first lets load up some sensor readings from an Arduino IoT node!

In [6]:
import numpy as np
import csv

# Sensor parameters
sample_period = 1  # ms
recording_length = 1e4  # ms
n_samples = recording_length / sample_period

ArduinoSensorsCSV = open("ArduinoSensors.csv")
sensor_recording = csv.reader(ArduinoSensorsCSV)
sensor_data = np.array([dataread for dataread in sensor_recording])